In [1]:
import math 
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
ufc_df = pd.read_excel("data_i/i_ufc_grouped_data_target.xlsx")

In [3]:
ufc_df.columns

Index(['Fighter', 'KD', 'SUB_ATT', 'SIG_STR_pct', 'REV', 'SIG_STR._lan',
       'SIG_STR._att', 'TOTAL_STR._lan', 'TOTAL_STR._att', 'TD_lan', 'TD_att',
       'HEAD_lan', 'HEAD_att', 'BODY_lan', 'BODY_att', 'LEG_lan', 'LEG_att',
       'DISTANCE_lan', 'DISTANCE_att', 'CLINCH_lan', 'CLINCH_att',
       'GROUND_lan', 'GROUND_att', 'pct_SIG_STR', 'pct_TOTAL_STR', 'pct_TD',
       'pct_HEAD', 'pct_BODY', 'pct_LEG', 'pct_DISTANCE', 'pct_CLINCH',
       'pct_GROUND', 'DIST_pct', 'CTRL_RT', 'SLpM', 'Str_Acc', 'SApM',
       'Str_Def', 'TD_Avg', 'TD_Acc', 'TD_Def', 'Sub_Avg', 'CTRL', 'CTRLLD',
       'fight_time', 'avg_fight_time', 'DIST', 'Total_Fights', 'Win', 'Loss',
       'Draw', 'longest_win_streak', 'win_by_Decision_Majority',
       'win_by_Decision_Split', 'win_by_Decision_Unanimous', 'win_by_KO/TKO',
       'win_by_Submission', 'win_by_TKO_Doctor_Stoppage', 'Height_cms',
       'Reach_cms', 'Weight_lbs', 'R_age', 'division', 'label', 'lab_num'],
      dtype='object')

In [4]:
ufc_df.head(10)

,Fighter,KD,SUB_ATT,SIG_STR_pct,REV,SIG_STR._lan,SIG_STR._att,TOTAL_STR._lan,TOTAL_STR._att,TD_lan,...,win_by_KO/TKO,win_by_Submission,win_by_TKO_Doctor_Stoppage,Height_cms,Reach_cms,Weight_lbs,R_age,division,label,lab_num
0,Aalon Cruz,0,0,0.16,0,2,15,2,15,0,...,0,0,0,182.88,198.12,145.0,30.0,Lightweight Division,Distance Striking Heavy - Non dominant wrestli...,6
1,Aaron Brink,0,0,0.00,0,0,5,0,5,0,...,0,0,0,190.50,NaN,205.0,27.0,Heavyweight Division,Distance Striking Heavy - Non dominant wrestli...,6
2,Aaron Phillips,0,1,1.55,1,62,109,346,432,0,...,0,0,0,175.26,180.34,135.0,31.0,Bantamweight Division,Wrestling/Grappling Heavy - Non dominant wrest...,9
3,Aaron Riley,0,1,3.10,0,314,904,581,1190,8,...,0,0,0,172.72,175.26,155.0,32.0,Lightweight Division,Distance Striking Heavy - Dominant grappling/w...,5
4,Aaron Rosa,0,0,0.99,0,132,290,407,598,0,...,0,0,0,193.04,195.58,205.0,34.0,Light Heavyweight Division,Balanced - Non dominant wrestling/grappling,2
5,Aaron Simpson,6,4,6.13,0,454,860,800,1265,32,...,2,0,0,182.88,185.42,170.0,38.0,Welterweight Division,Balanced - Dominant wrestling/grappling,1
6,Aaron Wilkinson,0,0,0.00,1,0,1,0,1,1,...,0,0,0,177.80,NaN,155.0,26.0,Lightweight Division,Wrestling/Grappling Heavy - Dominant wrestling...,8
7,Abdul Razak Alhassan,6,0,4.01,1,179,371,197,395,2,...,4,0,0,177.80,185.42,170.0,35.0,Middleweight Division,Balanced - Non dominant wrestling/grappling,2
8,Abdul-Kerim Edilov,0,1,0.61,0,77,125,150,210,2,...,0,0,0,187.96,NaN,205.0,29.0,Light Heavyweight Division,Wrestling/Grappling Heavy - Dominant wrestling...,8
9,Abel Trujillo,3,3,4.03,0,273,662,370,767,15,...,3,1,0,172.72,177.80,155.0,33.0,Lightweight Division,Balanced fighter,3


In [5]:
ufc_df.isnull().sum()

Fighter         0
KD              0
SUB_ATT         0
SIG_STR_pct     0
REV             0
               ..
Weight_lbs     10
R_age          27
division        0
label           0
lab_num         0
Length: 65, dtype: int64

In [6]:
division_df = ufc_df.groupby('division')
division_mean = round(division_df.mean(), 2)

# iterate over each row in the dataframe
for index, row in ufc_df.iterrows():
    # get the fighter's division and the values for CTRL_RT, Height, Weight, and Reach
    division = row['division']
    ctrl_rt = row['CTRL_RT']
    height = row['Height_cms']
    weight = row['Weight_lbs']
    reach = row['Reach_cms']
    age = row['R_age']
    
    # check if any of the values are missing
    if pd.isna(ctrl_rt) or pd.isna(height) or pd.isna(weight) or pd.isna(reach):
        # if any of the values are missing, impute with the division mean for that column
        if pd.isna(ctrl_rt):
            ufc_df.at[index, 'CTRL_RT'] = division_mean.loc[division, 'CTRL_RT']
        if pd.isna(height):
            ufc_df.at[index, 'Height_cms'] = division_mean.loc[division, 'Height_cms']
        if pd.isna(weight):
            ufc_df.at[index, 'Weight_lbs'] = division_mean.loc[division, 'Weight_lbs']
        if pd.isna(reach):
            ufc_df.at[index, 'Reach_cms'] = division_mean.loc[division, 'Reach_cms']
        if pd.isna(age):
            ufc_df.at[index, 'R_age'] = division_mean.loc[division, 'R_age']    

C:\Users\aksha\AppData\Local\Temp\ipykernel_20536\466790076.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  division_mean = round(division_df.mean(), 2)


In [ ]:
# create a bar plot
division_mean.plot(kind='bar', figsize=(15, 8))

# set plot title and axis labels
plt.title('UFC Fighter Stats by Division')
plt.xlabel('Division')
plt.ylabel('Mean Value')

# show legend
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

# show plot
plt.show()

In [ ]:
# Create a 5 by 6 matrix of subplots
fig, axes = plt.subplots(nrows=5, ncols=13, figsize=(40, 25))
axes = axes.ravel()

# Create histograms for each numerical column
for i, col in enumerate(ufc_df.select_dtypes(include=['float64', 'int64']).columns):
    division_df[col].hist(alpha=0.5, ax=axes[i])
    axes[i].set_title(col)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')
    if i == 0:
        axes[i].legend(division_df.groups.keys())

# Remove any unused subplots
for i in range(len(ufc_df.select_dtypes(include=['float64', 'int64']).columns), len(axes)):
    fig.delaxes(axes[i])

# Adjust the spacing between subplots
fig.tight_layout()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer, StandardScaler

In [ ]:
ufc_df.columns

In [ ]:
num_ufc = ufc_df[['KD', 'SIG_STR_pct','SLpM', 'Str_Acc', 'SApM', 
          'Str_Def', 'DIST_pct','pct_HEAD', 'pct_BODY', 'pct_LEG',
          'pct_TD', 'TD_Avg', 'TD_Acc', 'CTRL_RT','TD_Def', 'Sub_Avg', 'SUB_ATT', 'REV',    
          'Height_cms', 'Weight_lbs','Reach_cms', 'R_age', 
          'Win', 'Loss', 'Draw', 'Total_Fights', 'fight_time', 'avg_fight_time',
          'longest_win_streak', 'win_by_Decision_Majority', 'win_by_Decision_Split',
          'win_by_Decision_Unanimous', 'win_by_KO/TKO', 'win_by_Submission',
          'win_by_TKO_Doctor_Stoppage']]
num_ufc = num_ufc.dropna()

print(len(num_ufc.columns))

In [ ]:
# assume num_ufc is a NumPy array or Pandas DataFrame with multiple columns
cols_to_exclude = ['KD', 'SUB_ATT', 'REV', 'Draw']  # list of column names to exclude
included_cols = [col for col in num_ufc.columns if col not in cols_to_exclude]  # select columns to include

In [ ]:
# num_ufc_trsf = PowerTransformer().fit_transform(num_ufc[included_cols])
# pca_ufc = pd.DataFrame(num_ufc_trsf, columns=num_ufc[included_cols].columns) 

# # concatenate excluded columns with transformed data
# excluded_cols = num_ufc[cols_to_exclude]
# pca_ufc = pd.concat([pca_ufc, excluded_cols], axis=1)

# Create a StandardScaler object
scaler = StandardScaler()

# Fit and transform the data using the StandardScaler
num_ufc_scaled = scaler.fit_transform(num_ufc)

# Create a PowerTransformer object
power_transformer = PowerTransformer()

# Fit and transform the scaled data using the PowerTransformer
num_ufc_trsf = power_transformer.fit_transform(num_ufc_scaled)

pca_ufc = pd.DataFrame(num_ufc_trsf, columns=num_ufc.columns)

In [ ]:
# Create a 5 by 6 matrix of subplots
fig, axes = plt.subplots(nrows=5, ncols=7, figsize=(40, 25))
axes = axes.ravel()

# Create histograms for each numerical column
for i, col in enumerate(pca_ufc.columns):
    pca_ufc[col].hist(alpha=0.5, ax=axes[i])
    axes[i].set_title(col)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')
    if i == 0:
        axes[i].legend(pca_ufc.keys())

# Remove any unused subplots
for i in range(len(ufc_df.select_dtypes(include=['float64', 'int64']).columns), len(axes)):
    fig.delaxes(axes[i])

# Adjust the spacing between subplots
# fig.tight_layout()

In [ ]:
pca_ufc.isnull().sum()

In [ ]:
pca_ufc.columns

In [ ]:
pca = PCA()
pca.fit(pca_ufc)

In [ ]:
plt.plot(pca.explained_variance_, 'b')
plt.plot(pca.explained_variance_, 'bo')

In [ ]:
first_component_loadings = list(pca.components_[0])
second_component_loadings = list(pca.components_[1])
third_component_loadings = list(pca.components_[2])

loading_scores = {'Component 1': first_component_loadings,
                 'Component 2': second_component_loadings,
                 'Component 3': third_component_loadings}

df_loading_scores = pd.DataFrame(loading_scores)
df_loading_scores.index = list(pca_ufc.columns)

df_loading_scores.to_excel('data_i/i_loading_scores.xlsx')

In [ ]:
pca.components_[1]

In [ ]:
component_scores = pca_ufc.dot(pca.components_.T[0])
component_scores1 = pca_ufc.dot(pca.components_.T[1])
component_scores2 = pca_ufc.dot(pca.components_.T[2])
component_scores3 = pca_ufc.dot(pca.components_.T[3])
component_scores_df = pd.DataFrame({'Component Score 1': component_scores,
                                   'Component Score 2': component_scores1,
                                   'Component Score 3': component_scores2,
                                   'Component Score 4': component_scores3,})
combined_df = pd.concat([num_ufc, component_scores_df], axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))  # set the figure size to 8x6 inches

ax.barh(pca_ufc.columns, pca.components_[0])
ax.set_xlabel('PCA Component 1')
ax.set_ylabel('Features')
ax.set_title('PCA Feature Importance')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))  # set the figure size to 8x6 inches

ax.barh(pca_ufc.columns, pca.components_[1])
ax.set_xlabel('PCA Component 2')
ax.set_ylabel('Features')
ax.set_title('PCA Feature Importance')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))  # set the figure size to 8x6 inches

ax.barh(pca_ufc.columns, pca.components_[2])
ax.set_xlabel('PCA Component 3')
ax.set_ylabel('Features')
ax.set_title('PCA Feature Importance')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))  # set the figure size to 8x6 inches

ax.barh(pca_ufc.columns, pca.components_[4])
ax.set_xlabel('PCA Component 4')
ax.set_ylabel('Features')
ax.set_title('PCA Feature Importance')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))  # set the figure size to 8x6 inches

ax.barh(pca_ufc.columns, pca.components_[5])
ax.set_xlabel('PCA Component 5')
ax.set_ylabel('Features')
ax.set_title('PCA Feature Importance')
plt.show()

In [ ]:
loading_scores_1 = pca.components_[0]
loading_scores_2 = pca.components_[1]
loading_scores_3 = pca.components_[2]

print(loading_scores_1)
print(loading_scores_2)
print(loading_scores_3)

In [ ]:
p4p_fighters = ['Alexander Volkanovski', 'Anderson Silva', 'Brandon Moreno', 'Charles Oliveira',
               'Conor McGregor', 'Demetrious Johnson', 'Dustin Poirier', 'Francis Ngannou',
               'Georges St-Pierre', 'Henry Cejudo', 'Israel Adesanya', 'Jon Jones', 'Jose Aldo',
               'Justin Gaethje', 'Kamaru Usman', 'Khabib Nurmagomedov', 'Stipe Miocic']

# Create a new dataframe with observations only for the listed fighters
new_df = ufc_df[ufc_df['Fighter'].isin(p4p_fighters)]

# Display the new dataframe
new_df.head()

In [ ]:
p4p_df = new_df[['Fighter', 'KD', 'SIG_STR_pct','SLpM', 'Str_Acc', 'SApM', 
          'Str_Def', 'DIST_pct','pct_HEAD', 'pct_BODY', 'pct_LEG',
          'pct_TD', 'TD_Avg', 'TD_Acc', 'CTRL_RT','TD_Def', 'Sub_Avg', 'SUB_ATT', 'REV',    
          'Height_cms', 'Weight_lbs','Reach_cms', 'R_age', 
          'Win', 'Loss', 'Draw', 'Total_Fights', 'fight_time', 'avg_fight_time',
          'longest_win_streak', 'win_by_Decision_Majority', 'win_by_Decision_Split',
          'win_by_Decision_Unanimous', 'win_by_KO/TKO', 'win_by_Submission',
          'win_by_TKO_Doctor_Stoppage']]

In [ ]:
# Calculate total fights by fighter
total_fights = p4p_df['Total_Fights'].values
fighters = p4p_df['Fighter'].values
fighters2 = ufc_df['Fighter'].values

# Create a bar plot
plt.figure(figsize=(12, 6))
plt.bar(fighters, total_fights)
plt.xticks(rotation=90)
plt.xlabel('Fighter')
plt.ylabel('Total Fights')
plt.title('Total Fights by Fighter')
plt.show()

In [ ]:
# Extract the relevant columns
sig_str_pct = p4p_df['SIG_STR_pct'].values
slpm = p4p_df['SLpM'].values

# Create a scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(slpm, sig_str_pct)
plt.xlabel('Significant Strikes Landed per Minute (SLpM)')
plt.ylabel('Significant Strike Accuracy (%)')
plt.title('Significant Strike Accuracy vs. SLpM')
plt.show()

In [ ]:
# Extract the relevant columns
win_by_ko_tko = p4p_df['win_by_KO/TKO'].sum()
win_by_submission = p4p_df['win_by_Submission'].sum()
win_by_decision_unanimous = p4p_df['win_by_Decision_Unanimous'].sum()
win_by_decision_split = p4p_df['win_by_Decision_Split'].sum()
win_by_decision_majority = p4p_df['win_by_Decision_Majority'].sum()

# Create a pie chart
labels = ['KO/TKO', 'Submission', 'Decision (Unanimous)', 'Decision (Split)', 'Decision (Majority)']
sizes = [win_by_ko_tko, win_by_submission, win_by_decision_unanimous, win_by_decision_split, win_by_decision_majority]
plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.title('Win Distribution by Finish Type')
plt.show()

In [ ]:
import seaborn as sns

# Create a box plot
plt.figure(figsize=(30, 20))
sns.boxplot(x='Weight_lbs', y='Reach_cms', data=ufc_df)
plt.xlabel('Weight Class (lbs)')
plt.ylabel('Reach (cm)')
plt.title('Reach Distribution by Weight Class')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Create a histogram
plt.figure(figsize=(8, 6))
plt.hist(ufc_df['avg_fight_time'], bins=20, edgecolor='black')
plt.xlabel('Fight Time (minutes)')
plt.ylabel('Frequency')
plt.title('Fight Time Distribution')
plt.show()


In [ ]:
# Select the striking and grappling metrics columns
metrics = ['KD', 'SIG_STR_pct', 'SLpM', 'Str_Acc', 'SApM', 'Str_Def', 'DIST_pct',
           'pct_HEAD', 'pct_BODY', 'pct_LEG', 'pct_TD', 'TD_Avg', 'TD_Acc', 'CTRL_RT',
           'TD_Def', 'Sub_Avg', 'SUB_ATT', 'REV']

# Calculate the mean values for each metric
mean_values = p4p_df[metrics].mean()

# Create a bar plot
plt.figure(figsize=(12, 6))
mean_values.plot(kind='bar')
plt.xlabel('Metrics')
plt.ylabel('Mean Value')
plt.title('Comparison of Striking and Grappling Metrics')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Get the fighters' names
fighters = p4p_df['Fighter']

# Get the mean values for each metric per fighter
mean_values = p4p_df[metrics].mean()

# Create an array for the x-axis positions
x = np.arange(len(metrics))

# Create the bar plot
plt.figure(figsize=(12, 6))
plt.bar(x, mean_values, align='center')
plt.xlabel('Metrics')
plt.ylabel('Mean Value')
plt.title('Comparison of Striking and Grappling Metrics by Fighter')
plt.xticks(x, metrics, rotation=45, ha='right')
plt.legend(fighters, loc='best')
plt.tight_layout()
plt.show()